In [13]:
# Pour créer et sauvegarder le pipeline
exec(open('../exportations/taf.py').read())

# Pour utiliser le pipeline sauvegardé plus tard
import pickle
import pandas as pd

# Charger le pipeline
with open('../Models/Hotel_Complete_Pipeline.pkl', 'rb') as file:
    pipeline = pickle.load(file)

# Utiliser sur de nouvelles données
nouvelles_donnees = pd.read_csv('../Data/Test/Donnees_test.csv')
predictions = pipeline.predict(nouvelles_donnees)
probabilities = pipeline.predict_proba(nouvelles_donnees)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Meilleur score: 0.8584107232154019
Prédictions (1 = Not_Canceled, 0 = Canceled): [1 0 1 ... 1 0 1]
Probabilités [Canceled, Not_Canceled]: [[0.05367266 0.94632734]
 [0.76908775 0.23091225]
 [0.05330118 0.94669882]
 [0.12411252 0.87588748]
 [0.0062581  0.9937419 ]]
